In [1]:
#Dataset downloaded from:: https://nijianmo.github.io/amazon/index.html

In [3]:
#!pip install tensorflow_hub

In [2]:
#!pip install tensorflow==1.15.4 --user

In [1]:
 import tensorflow_hub as hub

# Importing Libraries & Basic Preprocesing 

In [1]:
#import all the necessary packages.

from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
import time
import re
import os
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack

warnings.filterwarnings("ignore")

In [3]:
# loading the data using pandas' read_json file.
data = pd.read_json('tops_fashion.json')

In [4]:
data.head(3)

,sku,asin,product_type_name,formatted_price,author,color,brand,publisher,availability,reviews,large_image_url,availability_type,small_image_url,editorial_review,title,model,medium_image_url,manufacturer,editorial_reivew
0,None,B016I2TS4W,SHIRT,None,None,None,FNC7C,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Minions Como Superheroes Ironman Women's O Nec...,Minions Como Superheroes Ironman Long Sleeve R...,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
1,None,B01N49AI08,SHIRT,None,None,None,FIG Clothing,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Sizing runs on the small side. FIG® recommends...,FIG Clothing Womens Izo Tunic,None,https://images-na.ssl-images-amazon.com/images...,None,NaN
2,None,B01JDPCOHO,SHIRT,None,None,None,FIG Clothing,None,None,"[False, https://www.amazon.com/reviews/iframe?...",https://images-na.ssl-images-amazon.com/images...,None,https://images-na.ssl-images-amazon.com/images...,Sizing runs on the small side. FIG® recommends...,FIG Clothing Womens Won Top,None,https://images-na.ssl-images-amazon.com/images...,None,NaN


In [5]:
#data.to_csv('amazon_apparel.csv')

In [6]:
print('='*80)
print("Number of data points in data=", data.shape[0])
print('-'*80)
print("Number of Columns in data=", data.shape[1])
print('-'*80)
print("The attributes(columns or features) of data =\n", data.columns.values)
print('='*80)

Number of data points in data= 183138
--------------------------------------------------------------------------------
Number of Columns in data= 19
--------------------------------------------------------------------------------
The attributes(columns or features) of data =
 ['sku' 'asin' 'product_type_name' 'formatted_price' 'author' 'color'
 'brand' 'publisher' 'availability' 'reviews' 'large_image_url'
 'availability_type' 'small_image_url' 'editorial_review' 'title' 'model'
 'medium_image_url' 'manufacturer' 'editorial_reivew']


`1. asin ( Amazon standard identification number)`
`2. brand ( brand to which the product belongs to )` 
`3. color ( Color information of apparel, it can contain many colors as a value ex: red and black stripes )`
`4. product_type_name (type of the apperal, ex: SHIRT/TSHIRT )`
`5. medium_image_url ( url of the image ) `
`6. title (title of the product.) `
`7. formatted_price (price of the product)`

In [7]:
data = data[['asin', 'brand', 'color', 'medium_image_url', 'product_type_name', 'title', 'formatted_price']]

In [8]:
data.head(3)

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,B016I2TS4W,FNC7C,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,Minions Como Superheroes Ironman Long Sleeve R...,None
1,B01N49AI08,FIG Clothing,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,FIG Clothing Womens Izo Tunic,None
2,B01JDPCOHO,FIG Clothing,None,https://images-na.ssl-images-amazon.com/images...,SHIRT,FIG Clothing Womens Won Top,None


In [9]:
data.isna().sum()

asin                      0
brand                   151
color                118182
medium_image_url          0
product_type_name         0
title                     0
formatted_price      154743
dtype: int64

In [10]:
data.isna().sum()/len(data)*100

asin                  0.000000
brand                 0.082451
color                64.531665
medium_image_url      0.000000
product_type_name     0.000000
title                 0.000000
formatted_price      84.495299
dtype: float64

In [11]:
#data=data[['asin', 'brand', 'medium_image_url', 'product_type_name', 'title']]

In [12]:
# consider products which have color information
# data['color'].isnull() => gives the information about the dataframe row's which have null values price == None|Null
data =data.loc[~data['color'].isnull()]
print('='*80)
print('Number of data points After eliminating color=NULL :', data.shape[0])
print('='*80)

Number of data points After eliminating color=NULL : 64956


In [13]:
data = data.loc[~data['formatted_price'].isnull()]
print('='*80)
print('Number of data points After eliminating price=NULL :', data.shape[0])
print('='*80)

Number of data points After eliminating price=NULL : 28385


In [14]:
data.shape

(28385, 7)

In [15]:
data_sorted = data[data['title'].apply(lambda x: len(x.split())>4)]
print('='*80)
print("After removal of products with short description:", data_sorted.shape[0])
print('='*80)

After removal of products with short description: 27949


In [16]:
# Sort the whole data based on title (alphabetical order of title) 
data_sorted.sort_values('title',inplace=True, ascending=False)
data_sorted.tail(3)

,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
109599,B00KI3VDXM,Crazy4Bling,Purple,https://images-na.ssl-images-amazon.com/images...,SHIRT,"""I Wanna Be Adored"" Long Sleeve Top with Shred...",$39.99
78827,B003IDE8XQ,Maggie's Organics,Grey,https://images-na.ssl-images-amazon.com/images...,HOME,"""Camisoles Grey - Medium Fair Labor, 1 pc""",$18.99
118987,B008D30AGK,Out+of+Print+Clothing,Multicolored,https://images-na.ssl-images-amazon.com/images...,SHIRT,"""1984"" Retro Book Cover Women's SLim Fit T-Shi...",$7.51


In [17]:
data_sorted.shape

(27949, 7)

In [18]:
data_1=data_sorted.sample(13000)

In [19]:
data_2=data_sorted.sample(13000)

In [20]:
data_1.shape

(13000, 7)

In [21]:
data_2.shape

(13000, 7)

In [22]:
#df.to_excel("output.xlsx")

In [23]:
#data_1.to_excel('amazon_apparel_sheet_1.xlsx')

In [24]:
#data_2.to_excel('amazon_apparel_sheet_2.xlsx')

In [2]:
data_1=pd.read_excel("amazon_apparel_sheet_1.xlsx")
data_1.head(2)

,Unnamed: 0,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,175113,B07175QZVB,Eileen Fisher,Catalina,https://images-na.ssl-images-amazon.com/images...,SHIRT,"Eileen Fisher Solid Boatneck Top, Size-XL",$79.00
1,28723,B01MSZ1EO7,Esprit,110 Off White,https://images-na.ssl-images-amazon.com/images...,SWEATER,Esprit Women's White Melange Women's Printed J...,$109.97


In [3]:
data_2=pd.read_excel("amazon_apparel_sheet_2.xlsx")
data_2.head(2)

,Unnamed: 0,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,85682,B06XPG6DMG,Head Case Designs,She Leaves Sparkle on White,https://images-na.ssl-images-amazon.com/images...,SHIRT,Official Tangerine-Tane She Leaves Sparkle Art...,$28.45
1,126443,B01GC5WSK6,BRMWs,Black-1,https://images-na.ssl-images-amazon.com/images...,BOOKS_1973_AND_LATER,Arsenal F.C. Logo Herbert Chapman Women 3/4 Sl...,$10.80


# Text Prepocessing & Vectorization of Text Data

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\DR SNEHAL
[nltk_data]     BANKAR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# we use the list of stop words that are downloaded from nltk lib.
stop_words = set(stopwords.words('english'))
print ('list of stop words:', stop_words)

def nlp_preprocessing(total_text, index, column,df):
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            # remove the special chars in review like '"#$@!%^&*()_+-~?>< etc.
            word = ("".join(e for e in words if e.isalnum()))
            # Conver all letters to lower-case
            word = word.lower()
            # stop-word removal
            if not word in stop_words:
                string += word + " "
        df[column][index] = string

list of stop words: {'very', 'the', 'some', 'itself', 'who', 'now', 've', 'can', 'isn', 'shouldn', 'below', "needn't", 'had', 'me', 'm', 'from', 'hers', 'you', 'into', 'just', "couldn't", "you'll", "doesn't", 'ma', 'am', 'or', 'if', 'shan', 'out', "shouldn't", 'won', 'no', 'themselves', 'for', 'with', 'should', "she's", 'herself', 'in', 'why', 'those', 'their', 's', 'y', "shan't", 'then', 'after', 'haven', 'being', "weren't", 'under', 'too', 'will', 'having', 'our', 'ain', 'more', "hadn't", 'do', 'during', 'once', 'them', 'his', "isn't", "mustn't", 'both', "wouldn't", "hasn't", 'are', 'been', 'a', 'have', 'up', 'here', 'an', 'd', 'couldn', "it's", 'on', "aren't", "wasn't", "you've", 'where', 'as', "should've", 'yourselves', 'its', 'whom', 'wouldn', 'her', 'of', 'between', 'against', 'than', 'how', 'until', 'each', 'o', 'over', 'not', 'but', 'hadn', 'such', 'doesn', 'yours', 'does', 'by', 'mightn', 're', 'has', 'll', 'few', "didn't", 'that', 'my', 't', 'was', 'again', 'down', 'ours', 'd

In [6]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in data_1.iterrows():
    nlp_preprocessing(row['title'], index, 'title',data_1)
# we print the time it took to preprocess whole titles 
print(time.clock() - start_time, "seconds")

6.466201799999993 seconds


In [7]:
data_1['title'].head(2)

0             eileen fisher solid boatneck top sizexl 
1    esprit womens white melange womens printed jum...
Name: title, dtype: object

In [8]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in data_2.iterrows():
    nlp_preprocessing(row['title'], index, 'title',data_2)
# we print the time it took to preprocess whole titles 
print(time.clock() - start_time, "seconds")

6.5052797999999825 seconds


In [9]:
data_2['title'].head(2)

0    official tangerinetane leaves sparkle art xl  ...
1    arsenal fc logo herbert chapman women 34 sleev...
Name: title, dtype: object

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
title_vectorizer = CountVectorizer()
title_vectorizer.fit((data_1['title'].values)+(data_2['title'].values))
title_features_1=title_vectorizer.transform(data_1['title'].values)
title_features_1.get_shape() # get number of rows and columns in feature matrix.

(13000, 11767)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
#title_vectorizer = CountVectorizer(max_features=title_features_1.get_shape()[1])
title_features_2   = title_vectorizer.transform(data_2['title'].values)
title_features_2.get_shape() # get number of rows and columns in feature matrix.

(13000, 11767)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
pairwise_dist = cosine_similarity(title_features_1[100],title_features_2[100])

In [14]:
pairwise_dist.shape

(1, 1)

In [15]:
np.nonzero(pairwise_dist)

(array([], dtype=int64), array([], dtype=int64))

In [16]:
#data_2['title'][165580]

In [17]:
np.amax(pairwise_dist)

0.0

In [18]:
def bag_of_words_model(doc_id, num_results):
    # doc_id: apparel's id in given corpus
    
    # pairwise_dist will store the distance from given input apparel to all remaining apparels
    # the metric we used here is cosine, the coside distance is mesured as K(X, Y) = <X, Y> / (||X||*||Y||)
    # http://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
    print("Title from dataset 2:",data_2.iloc[doc_id]['title'])
    print('='*60)
    pairwise_dist = cosine_similarity(title_features_1,title_features_2[doc_id])
    
    # np.argsort will return indices of the smallest distances
    indices = np.argsort(pairwise_dist.flatten())[::-1][0:num_results]
    #pdists will store the smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[::-1][0:num_results]

    #data frame indices of the 9 smallest distace's
    df_indices = list(data_1.index[indices])
    
    for i in range(0,len(indices)):
        # we will pass 1. doc_id, 2. title1, 3. title2, url, model
        #get_result(indices[i],data['title'].loc[df_indices[0]], data['title'].loc[df_indices[i]], data['medium_image_url'].loc[df_indices[i]], 'bag_of_words')
        print('ASIN :',data_1['asin'].loc[df_indices[i]])
        print ('Brand:', data_1['brand'].loc[df_indices[i]])
        print ('Title:', data_1['title'].loc[df_indices[i]])
        print ('Cosine similarity with the query point',pdists[i])
        print('='*60)

In [19]:
data_1=data_1.reset_index(drop=True)

In [20]:
data_1[data_1['asin']=='B074JBMY4C'].index

Int64Index([1680], dtype='int64')

In [21]:
bag_of_words_model(400, 5)

Title from dataset 2: blousebaomabao women summer lace vest shirt casual tank tops medium 
ASIN : B01EMNLU90
Brand: Baomabao_Shirt
Title: blousebaomabao women summer lace vest shirt casual tank tops xlarge 
Cosine similarity with the query point 0.8999999999999999
ASIN : B01F852VDK
Brand: Perman
Title: perman women summer vest sleeveless shirt blouse casual tank tops tshirt l black 
Cosine similarity with the query point 0.6390096504226939
ASIN : B01J5G7NMW
Brand: Sumilulu
Title: sumilulu women summer vest sleeveless blouse casual tank tops tshirt xl 
Cosine similarity with the query point 0.5720775535473555
ASIN : B01E9T0ZX4
Brand: Overmal Top
Title: overmal women summer lace vest top short sleeve blouse tank tops tshirt l 
Cosine similarity with the query point 0.5477225575051662
ASIN : B01J5F5D5C
Brand: Sumilulu
Title: sumilulu fashion women summer vest top sleeveless blouse casual tank tops tshirt l 
Cosine similarity with the query point 0.5477225575051662


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
tfidf_title_vectorizer = TfidfVectorizer()
tfidf_title_vectorizer.fit((data_1['title'].values)+(data_2['title'].values))

title_feature_1_tfidf=tfidf_title_vectorizer.transform(data_1['title'].values)
title_feature_2_tfidf=tfidf_title_vectorizer.transform(data_2['title'].values)

In [24]:
def tfidf_model(doc_id, num_results):
    # doc_id: apparel's id in given corpus
    
    # pairwise_dist will store the distance from given input apparel to all remaining apparels
    # the metric we used here is cosine, the coside distance is mesured as K(X, Y) = <X, Y> / (||X||*||Y||)
    # http://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
    print("Title from dataset 2:",data_2.iloc[doc_id]['title'])
    print('='*60)
    pairwise_dist = cosine_similarity(title_feature_1_tfidf,title_feature_2_tfidf[doc_id])
    
    # np.argsort will return indices of the smallest distances
    indices = np.argsort(pairwise_dist.flatten())[::-1][0:num_results]
    #pdists will store the smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[::-1][0:num_results]

    #data frame indices of the 9 smallest distace's
    df_indices = list(data_1.index[indices])
    
    for i in range(0,len(indices)):
        # we will pass 1. doc_id, 2. title1, 3. title2, url, model
        #get_result(indices[i],data['title'].loc[df_indices[0]], data['title'].loc[df_indices[i]], data['medium_image_url'].loc[df_indices[i]], 'bag_of_words')
        print('ASIN :',data_1['asin'].loc[df_indices[i]])
        print ('Brand:', data_1['brand'].loc[df_indices[i]])
        print ('Title:', data_1['title'].loc[df_indices[i]])
        print ('Cosine similarity with the query point',pdists[i])
        print('='*60)

In [25]:
tfidf_model(400, 5)

Title from dataset 2: blousebaomabao women summer lace vest shirt casual tank tops medium 
ASIN : B01EMNLU90
Brand: Baomabao_Shirt
Title: blousebaomabao women summer lace vest shirt casual tank tops xlarge 
Cosine similarity with the query point 0.9267240987341656
ASIN : B01F852VDK
Brand: Perman
Title: perman women summer vest sleeveless shirt blouse casual tank tops tshirt l black 
Cosine similarity with the query point 0.37931874956633915
ASIN : B01E9T0ZX4
Brand: Overmal Top
Title: overmal women summer lace vest top short sleeve blouse tank tops tshirt l 
Cosine similarity with the query point 0.37758531649196314
ASIN : B01E9T0ZSE
Brand: Overmal Top
Title: overmal women summer lace vest top short sleeve blouse tank tops tshirt xl 
Cosine similarity with the query point 0.36559599980501256
ASIN : B01J5G7NMW
Brand: Sumilulu
Title: sumilulu women summer vest sleeveless blouse casual tank tops tshirt xl 
Cosine similarity with the query point 0.3525769471601655


In [32]:
data_1['brand'].fillna(value="Not given", inplace=True)
data_2['brand'].fillna(value="Not given", inplace=True)

In [44]:
#data_2['brand'].nunique()

In [45]:
#data_2.head(

In [37]:
brands = [x.replace(" ", "-") for x in data_1['brand'].values + data_2['brand'].values]
types = [x.replace(" ", "-") for x in data_1['product_type_name'].values + data_2['product_type_name'].values]
colors = [x.replace(" ", "-") for x in data_1['color'].values + data_2['color'].values]

In [38]:
brand_vectorizer = CountVectorizer()
brand_features = brand_vectorizer.fit_transform(brands)

type_vectorizer = CountVectorizer()
type_features = type_vectorizer.fit_transform(types)

color_vectorizer = CountVectorizer()
color_features = color_vectorizer.fit_transform(colors)

extra_features = hstack((brand_features, type_features, color_features)).tocsr()

In [41]:
def tfidf_weighted_other_feature_model(doc_id, w1, w2, num_results):
    # doc_id: apparel's id in given corpus
    
    # pairwise_dist will store the distance from given input apparel to all remaining apparels
    # the metric we used here is cosine, the coside distance is mesured as K(X, Y) = <X, Y> / (||X||*||Y||)
    # http://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
    print("Title from dataset 2:",data_2.iloc[doc_id]['title'])
    print('='*60)
    tfidf_dist = cosine_similarity(title_feature_1_tfidf,title_feature_2_tfidf[doc_id])
    ex_feat_dist = cosine_similarity(extra_features, extra_features[doc_id])
    pairwise_dist   = (w1 * tfidf_dist +  w2 * ex_feat_dist)/float(w1 + w2)
    
    # np.argsort will return indices of the smallest distances
    indices = np.argsort(pairwise_dist.flatten())[::-1][0:num_results]
    #pdists will store the smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[::-1][0:num_results]

    #data frame indices of the 9 smallest distace's
    df_indices = list(data_1.index[indices])
    
    for i in range(0,len(indices)):
        # we will pass 1. doc_id, 2. title1, 3. title2, url, model
        #get_result(indices[i],data['title'].loc[df_indices[0]], data['title'].loc[df_indices[i]], data['medium_image_url'].loc[df_indices[i]], 'bag_of_words')
        print('ASIN :',data_1['asin'].loc[df_indices[i]])
        print ('Brand:', data_1['brand'].loc[df_indices[i]])
        print ('Title:', data_1['title'].loc[df_indices[i]])
        print ('Cosine similarity with the query point',pdists[i])
        print('='*60)

In [43]:
tfidf_weighted_other_feature_model(12566, 5, 20, 5)

Title from dataset 2: wine yoga pants bachelor tank top small black 
ASIN : B01N0DNI1C
Brand: Kylin Express
Title: elegant shirt collar neckband detachable false collar stand collar fake collar yellow 
Cosine similarity with the query point 0.8
ASIN : B01N9K7981
Brand: Kylin Express
Title: detachable false collar necklace elegant shirt collar fake collar stand collar 06 
Cosine similarity with the query point 0.5366563145999496
ASIN : B074LD6NF7
Brand: KpopBaby
Title: kpopbaby womens long sleeve knitted casual cardigan outwear coats sweater tops yellow 
Cosine similarity with the query point 0.46188021535170065
ASIN : B0713TJWGP
Brand: TBONTB
Title: 34 sleeve tunic top women small mustard yellow 
Cosine similarity with the query point 0.4125454416727926
ASIN : B00P1OVV9E
Brand: Emmas Closet
Title: womens long sleeve bamboo top loose fit tunic medium yellow 
Cosine similarity with the query point 0.40298019051652495


In [3]:
#!pip install tensorflow